In [1]:
import numpy as np
from python_vehicle_simulator.vehicles import otter
from python_vehicle_simulator.lib import *

In [ ]:
vessel = otter

In [2]:
def PID(
    e_int,
    e_x,
    e_v,
    x_d,
    v_d,
    a_d,
    Kp,
    Ki,
    Kd,
    wn_d,
    zeta_d,
    r,
    v_max,
    sampleTime,
):

    # PID control law
    u = -Kp * e_x - Kd * e_v - Ki * e_int

    # Integral error, Euler's method
    e_int += sampleTime * e_x

    # 3rd-order reference model for smooth position, velocity and acceleration
    [x_d, v_d, a_d] = refModel3(x_d, v_d, a_d, r, wn_d, zeta_d, v_max, sampleTime)

    return u, e_int, x_d, v_d, a_d